In [1]:
%gui qt6

# Qt6 stuff
from PyQt6.QtWidgets import QApplication
from PyQt6 import QtWidgets
from PyQt6 import QtGui
from PyQt6 import QtCore
from PyQt6.QtCore import pyqtSignal

# Plotting stuff
import pyqtgraph as pg
import matplotlib.pyplot as plt
import cmasher as cmr

# Science stuff
import numpy as np
import pandas as pd
from spectres import spectres

# Astropy stuff
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.units.quantity import Quantity
from astropy.io.ascii import read as ascii_read

# zHunter stuff
from zhunter import DIRS
from zhunter import io
from zhunter.misc import set_up_linked_vb
from zhunter.colors import get_gradient
from zhunter.spectroscopic_system import SpecSystemModel, SpecSystem

# General stuff
import logging
import sys
from pathlib import Path
from itertools import cycle

In [2]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance
log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s [%(name)s] %(message)s')
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)

colors = cmr.take_cmap_colors('Spectral', 10, cmap_range=(0.0, 1), return_fmt='hex')

## Generate fake data

In [3]:
# The fake data
N_data = 10000
N_spat = 40
wvlg = np.linspace(300, 900, N_data)
spat = np.linspace(-5, 5, N_spat)
flux = np.ones((wvlg.shape[0], spat.shape[0])) * np.exp(-(spat)**2/0.1)
flux += np.random.normal(0, 0.1, size=(wvlg.shape[0], spat.shape[0]))
flux = flux.T
wvlg_tell = wvlg
wvlg_sky_bkg = wvlg
tellurics = 1 - np.exp(-(wvlg-600)**2/(30)**2)
sky_bkg = np.exp(-(wvlg-800)**2/(10)**2)


## Or real data

In [4]:
fname_2D = Path('/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/XSHOOTER_esoreflex_2D.fits.gz')
fname_telluric = Path('/Users/palmerio/Code_projects/zHunter/src/zhunter/data/tellurics/synth_tellurics_350_1100nm.csv.gz')
fname_sky_bkg = Path('/Users/palmerio/Code_projects/zHunter/src/zhunter/data/sky_background/sky_bkg_norm_nir_9000_23000.csv.gz')

wvlg, spat, flux, unc = io.read_fits_2D_spectrum(fname_2D)
wvlg = wvlg.value
spat = spat.value
flux = flux.value
unc = unc.value

spec_tell = io.read_generic_1D_spectrum(fname_telluric)
wvlg_tell = spec_tell.spectral_axis.value
tellurics = spec_tell.flux.value

spec_sky_bkg = io.read_generic_1D_spectrum(fname_sky_bkg)
wvlg_sky_bkg = spec_sky_bkg.spectral_axis.value
sky_bkg = spec_sky_bkg.flux.value



2023-03-01 14:18:59,322 DEBUG [zhunter.io] Attempting to read file: /Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/XSHOOTER_esoreflex_2D.fits.gz
2023-03-01 14:19:00,081 WARNING [astropy] UnitsWarning: 'erg/s/cm2/Angstrom' contains multiple slashes, which is discouraged by the FITS standard
2023-03-01 14:19:00,097 DEBUG [zhunter.io] Using the FITS CD matrix.
2023-03-01 14:19:00,098 DEBUG [zhunter.io] PIX=1.0 VAL=-10.58000019073486 DELT=0.159999847412109
2023-03-01 14:19:00,099 DEBUG [zhunter.io] Using the FITS CD matrix.
2023-03-01 14:19:00,100 DEBUG [zhunter.io] PIX=1.0 VAL=533.66 DELT=0.0199999999999818
2023-03-01 14:19:00,104 DEBUG [zhunter.io] Attempting to read file: /Users/palmerio/Code_projects/zHunter/src/zhunter/data/tellurics/synth_tellurics_350_1100nm.csv.gz


2023-03-01 14:19:00,399 DEBUG [zhunter.io] Did not find any name matching ['ERR', 'NOISE', 'SIGMA', 'UNC'] in ['wave(AA)', 'flux_norm']
2023-03-01 14:19:00,400 DEBUG [zhunter.io] Parsed unit AA from column name 'wave(AA)'
2023-03-01 14:19:00,401 DEBUG [zhunter.io] No unit could be parsed from column name 'flux_norm'
2023-03-01 14:19:00,402 INFO [zhunter.io] No units specified for flux, assuming ADU.
2023-03-01 14:19:00,425 WARNING [zhunter.io] No error/uncertainty found in file /Users/palmerio/Code_projects/zHunter/src/zhunter/data/tellurics/synth_tellurics_350_1100nm.csv.gz.
2023-03-01 14:19:00,427 DEBUG [zhunter.io] Attempting to read file: /Users/palmerio/Code_projects/zHunter/src/zhunter/data/sky_background/sky_bkg_norm_nir_9000_23000.csv.gz
2023-03-01 14:19:00,471 DEBUG [zhunter.io] Did not find any name matching ['ERR', 'NOISE', 'SIGMA', 'UNC'] in ['wave(AA)', 'flux_norm']
2023-03-01 14:19:00,472 DEBUG [zhunter.io] Parsed unit AA from column name 'wave(AA)'
2023-03-01 14:19:00,47

## Create the Widgets that will hold the plots

In [5]:
graphLayout = pg.GraphicsLayoutWidget(show=True)

In [20]:
# Clear everything
graphLayout.clear()
telluric_vb.clear()
sky_bkg_vb.clear()

NameError: name 'telluric_vb' is not defined

In [6]:
## Colors
from zhunter.colors import COLORS
color_style = 'kraken'
colors = COLORS[color_style]
# colors

In [7]:
pg.setConfigOption("foreground", colors['foreground'])
pg.setConfigOption("background", colors['background'])


In [8]:
# The plotting layout
graphLayout.ci.setBorder((50, 50, 100)) # this is to see where the Items' bounds are
graphLayout.addLabel('File Name', row=0, colspan=2)
ax2D = graphLayout.addPlot(row=1, col=0)
ax1D = graphLayout.addPlot(row=2, col=0)
ax2D_side_vb = graphLayout.addViewBox(row=1, col=1)
img_hist = pg.HistogramLUTItem(gradientPosition="left")
graphLayout.addItem(img_hist, row=2, col=1)
telluric_vb = set_up_linked_vb(ax1D)
sky_bkg_vb =  set_up_linked_vb(ax1D)

# Add empty legend
legend = pg.LegendItem(offset=(-10,10))
legend.setParentItem(ax1D)
ax1D.legend = legend


In [9]:
# Change the ratios of sizes of PlotItems
graphLayout.ci.layout.setRowStretchFactor(1, 2)
graphLayout.ci.layout.setRowStretchFactor(2, 3)
# Strech column 0 (where 1D and 2D plots are) to make it bigger in x than the side histograms
graphLayout.ci.layout.setColumnStretchFactor(0, 100)
# Link the axis together
ax2D.hideAxis("bottom")
ax1D.getAxis('left').setWidth(60)
ax2D.getAxis('left').setWidth(60)
ax1D.vb.setXLink(ax2D.vb)
ax2D_side_vb.setYLink(ax2D.vb)

In [10]:
# Set limits
ax1D.vb.setLimits(
            xMin=np.min(wvlg), xMax=np.max(wvlg)
        )
ax2D.vb.setLimits(
                xMin=np.min(wvlg),
                xMax=np.max(wvlg),
                yMin=np.min(spat),
                yMax=np.max(spat),
            )
ax2D_side_vb.setLimits(
                yMin=np.min(spat), yMax=np.max(spat)
            )


### Add data to visualize

In [11]:
# Create image item, set the image to the flux array
# and add it to the 2D PlotItem
img = pg.ImageItem()
img.setImage(flux.T)
ax2D.addItem(img)
# Transform image indexes to physical coordinates
rect = QtCore.QRectF(wvlg[0], spat[0], 
                     wvlg[-1] - wvlg[0], spat[-1] - spat[0])
img.setRect(rect)
cmap = pg.colormap.get('afmhot',source='matplotlib', skipCache=True)
img.setColorMap(cmap)
# LUT
img_hist.setImageItem(img)
img_hist.gradient.setColorMap(cmap)
img_hist.setHistogramRange(np.quantile(flux, 0.025), np.quantile(flux, 0.975))
img_hist.setLevels(np.quantile(flux, 0.025), np.quantile(flux, 0.975))

# Create 1D spec
flux_1D_spec = pg.PlotCurveItem(
    np.zeros(2),
    np.zeros(1),
    pen=pg.mkPen(color=colors['spec']),
    stepMode='center')
ax1D.addItem(flux_1D_spec)

# Region of Interest
spatial_width = 1
spat_med = np.median(spat)
roi = pg.ROI([np.min(wvlg), spat_med - spatial_width/2], [np.max(wvlg)-np.min(wvlg), spatial_width],
             pen=pg.mkPen('g', width=3))
ax2D.addItem(roi)
roi.setZValue(10)

flux_selected = roi.getArrayRegion(flux.T,
                                   img,
                                   returnMappedCoords=True)

flux_1D = flux_selected[0].mean(axis=1)
wvlg_1D = flux_selected[1][0,:,0]
dx = wvlg_1D[1] - wvlg_1D[0]
wvlg_bins = list(wvlg_1D) + [wvlg_1D[-1]+dx]

flux_1D_spec.setData(wvlg_bins, flux_1D)

# Sideview hist
sidehist_2D = pg.PlotCurveItem(
    np.zeros(1),
    np.zeros(1),
    pen=pg.mkPen(color=colors['foreground'])
)
y_dist = np.median(flux, axis=1)
sidehist_2D.setData(y_dist, spat)

ax2D_side_vb.addItem(sidehist_2D)

# Add tellurics

telluric_pi = pg.PlotCurveItem(
            wvlg_tell,
            tellurics,
            pen=pg.mkPen(QtGui.QColor(colors['sky']), width=0.3),
            brush=QtGui.QBrush(get_gradient(QtGui.QColor(colors['sky']))),
            fillLevel=1,
        )
telluric_vb.addItem(telluric_pi)

# Add sky background
sky_bkg_pi = pg.PlotCurveItem(
            wvlg_sky_bkg,
            sky_bkg,
            pen=pg.mkPen(QtGui.QColor(colors['sky']), width=0.3),
            brush=QtGui.QBrush(get_gradient(QtGui.QColor(colors['sky']), reverse=True)),
            fillLevel=0,
        )
sky_bkg_vb.addItem(sky_bkg_pi)
ax1D.showGrid(x=True, y=True)

## Spectroscopic Systems

In [12]:
# Abstract Model to hold the list of spectroscopic systens
ssm = SpecSystemModel()


# QListView widget to view the model
specsysView = QtWidgets.QListView()
specsysView.setModel(ssm)
specsysView.show()

# Main window in which to embed the QListView widget
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("My App")
        widget = specsysView
        self.setCentralWidget(widget)
win = MainWindow()
win.show()

In [13]:
# Spectroscopic system
specsys = SpecSystem(6.318, PlotItem=ax1D, sys_type='abs',
                     color='cyan')
specsys.draw(xmin=np.min(wvlg)*u.nm, xmax=np.max(wvlg)*u.nm,)

ssm.specsystems.append((True, specsys))
specsys.edited.connect(ssm.layoutChanged.emit)
ssm.sort()
ssm.layoutChanged.emit()

2023-03-01 14:19:12,729 DEBUG [zhunter.spectroscopic_system] Drawing abs system at redshift : 6.31800


## Velocity plot

In [14]:
from astropalmerio.spectra.conversions import *
from itertools import product

In [34]:
velLayout = pg.GraphicsLayoutWidget(show=True, border=(100,100,100))

qt.pointer.dispatch: skipping QEventPoint(id=0 ts=0 pos=0,0 scn=1243.24,694.147 gbl=1243.24,694.147 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1243.24,-694.147 last=-1243.24,-694.147 Δ 1243.24,694.147) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1169.26,711.428 gbl=1169.26,711.428 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1169.26,-711.428 last=-1169.26,-711.428 Δ 1169.26,711.428) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=2 ts=0 pos=0,0 scn=1135.96,771.005 gbl=1135.96,771.005 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1135.96,-771.005 last=-1135.96,-771.005 Δ 1135.96,771.005) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=3 ts=0 pos=0,0 scn=1158.28,708.726 gbl=1158.28,708.726 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1158.28,-708.726 last=-1158.28,-708.726 Δ 1158.28,708.726) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1211.88,233.44 gbl=1211.88,233.44 Released ellipse=(1x1 

In [53]:
velLayout.clear()

### Define lines to plot

In [29]:
lines = ascii_read(DIRS["DATA"] / "lines/basic_line_list.ecsv")
lnames = ['SiII_1260', 'OI_1302', 'SiII_1304']#, 'CII_1334', 'SiIV_1394']
z = 6.317
waves = np.array([l['wave'] for l in lines if l['name'] in lnames]) * lines['wave'].unit
waves_obs = waves.to('nm') * (1+z)

In [31]:
from zhunter.CheckBoxListWidget import CheckBoxListWidget
# Create checkboxlist widget
linelistView = CheckBoxListWidget()
linelistView.show()
linelistView.addItems(lines['name'])


In [46]:
lnames = []
for row in linelistView.getCheckedRows():
     lnames.append(linelistView.item(row).text())

lines_to_plot = {
            n: {"name": n, "rest_wave": rw, "obs_wave": ow}
            for n, rw, ow in zip(lnames, waves, waves_obs)
        }
lines_to_plot


{'SiII_1304': {'name': 'SiII_1304',
  'rest_wave': <Quantity 1302.1685 Angstrom>,
  'obs_wave': <Quantity 952.79669145 nm>},
 'SiII_1260': {'name': 'SiII_1260',
  'rest_wave': <Quantity 1304.3702 Angstrom>,
  'obs_wave': <Quantity 954.40767534 nm>}}

In [54]:
n_l = len(lines_to_plot)
if n_l <= 4:
    n_cols = 1
    n_rows = n_l
else:
    n_cols = int(n_l/2)
    n_rows = int(n_l/2)
    if n_l % 2 != 0:
        n_rows += 1

vel_min = -600 * u.Unit('km/s')
vel_max = 600 * u.Unit('km/s')
ref_lname = lnames[0]
for lines, indexes in zip(lines_to_plot.items(), product(range(1,n_cols+1), range(n_rows))):
    i, j = indexes
    lname, line = lines
    vel = wave_to_vel(wvlg_bins*u.nm, line['obs_wave'])
    imin = vel.searchsorted(vel_min)
    imax = vel.searchsorted(vel_max)
    vel_1D_spec = pg.PlotCurveItem(
        vel.value,
        flux_1D,
        pen=pg.mkPen(color=colors['spec']),
        stepMode='center')
    line['pi'] = velLayout.addPlot(title=lname.replace('_',' '), name=lname, col=i, row=j)
    line['pi'].addItem(vel_1D_spec)
    line['pi'].showGrid(x=True, y=True)
    line['pi'].hideButtons()
    line['pi'].setXLink(ref_lname)
    line['pi'].setXRange(-600, 600)
    line['pi'].setYRange(np.min(flux_1D[imin:imax]),
                         np.max(flux_1D[imin:imax]))
    line['pi'].vb.setLimits(
            xMin=np.min(vel.value), xMax=np.max(vel.value)
        )

velLayout.addLabel('Velocity (km/s)', col=1, row=n_rows+1, colspan=n_cols)
velLayout.addLabel('Flux', angle=-90, col=0, rowspan=n_rows)


In [52]:
line['pi'].vb.state

{'targetRange': [[-677.4596669241483, 677.4596669241483],
  [-7.016480923970568e-16, 1.4680171089385755e-15]],
 'viewRange': [[-677.4596669241483, 677.4596669241483],
  [-7.016480923970568e-16, 1.4680171089385755e-15]],
 'yInverted': False,
 'xInverted': False,
 'aspectLocked': False,
 'autoRange': [False, True],
 'autoPan': [False, False],
 'autoVisibleOnly': [False, False],
 'linkedViews': ['SiII_1304', None],
 'defaultPadding': 0.02,
 'mouseEnabled': [True, True],
 'mouseMode': 3,
 'enableMenu': True,
 'wheelScaleFactor': -0.125,
 'background': None,
 'logMode': [False, False],
 'limits': {'xLimits': [-132162.5789975226, 20603.443102673613],
  'yLimits': [-1e+307, 1e+307],
  'xRange': [None, None],
  'yRange': [None, None]}}

In [55]:
linelistView.clear()

In [3]:
from astropy.io.ascii import read as ascii_read


In [19]:
tab = ascii_read('/Users/palmerio/Code_projects/zHunter/src/zhunter/data/lines/basic_line_list.csv')
tab

name,wave
str10,float64
Ly_alpha,1215.6701
Ly_beta,1025.7223
Ly_gamma,972.5368
Ly_delta,949.7431
Ly_limit,911.7633
CII_1334,1334.5323
CII_1335*,1335.7077
CIV_1548,1548.2049
CIV_1550,1550.77845


In [26]:
tab['wave'].unit = u.Unit('Angstrom')

In [27]:
tab['wave'].unit

Unit("Angstrom")